<a href="https://colab.research.google.com/github/guifav/marco_energee/blob/main/Marco_Energee_Estrutura_de_dados_para_contas_de_Energia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Criei uma estrutura abrangente que permita armazenar e organizar todas as informações das contas de energia de forma normalizada.

Esta estrutura foi projetada para:

- Acomodar diferentes padrões de distribuidoras
- Permitir análises detalhadas de consumo e custos
- Manter histórico completo de faturamento
- Facilitar a integração com outros sistemas
- Garantir a rastreabilidade das informações

# Estrutura de Dados para Contas de Energia

## 1. Tabelas Principais

### 1.1 DISTRIBUIDORA
```sql
CREATE TABLE DISTRIBUIDORA (
    id_distribuidora    INTEGER PRIMARY KEY,
    nome               VARCHAR(100),
    cnpj               VARCHAR(14),
    inscricao_estadual VARCHAR(20)
);
```

### 1.2 UNIDADE_CONSUMIDORA
```sql
CREATE TABLE UNIDADE_CONSUMIDORA (
    id_unidade         INTEGER PRIMARY KEY,
    numero_cliente     VARCHAR(20),
    id_distribuidora   INTEGER,
    grupo_subgrupo     VARCHAR(10),
    classe_subclasse   VARCHAR(100),
    tipo_fornecimento  VARCHAR(50),
    tensao_nominal     VARCHAR(20),
    filial_mult        VARCHAR(10),
    filial_sap         VARCHAR(10),
    endereco           VARCHAR(200),
    cidade             VARCHAR(100),
    estado             VARCHAR(2),
    cep                VARCHAR(10),
    FOREIGN KEY (id_distribuidora) REFERENCES DISTRIBUIDORA(id_distribuidora)
);
```

### 1.3 FATURA
```sql
CREATE TABLE FATURA (
    id_fatura          INTEGER PRIMARY KEY,
    id_unidade         INTEGER,
    numero_fatura      VARCHAR(20),
    serie_fatura       VARCHAR(10),
    data_emissao      DATE,
    data_vencimento   DATE,
    data_leitura_atual DATE,
    data_leitura_anterior DATE,
    dias_faturados    INTEGER,
    valor_total       DECIMAL(15,2),
    chave_nfe         VARCHAR(100),
    protocolo_autorizacao VARCHAR(100),
    FOREIGN KEY (id_unidade) REFERENCES UNIDADE_CONSUMIDORA(id_unidade)
);
```

### 1.4 LEITURA
```sql
CREATE TABLE LEITURA (
    id_leitura        INTEGER PRIMARY KEY,
    id_fatura         INTEGER,
    leitura_anterior  INTEGER,
    leitura_atual     INTEGER,
    consumo_kwh       INTEGER,
    demanda_medida    DECIMAL(10,2),
    demanda_faturada  DECIMAL(10,2),
    FOREIGN KEY (id_fatura) REFERENCES FATURA(id_fatura)
);
```

## 2. Tabelas de Componentes Tarifários

### 2.1 CONSUMO_FATURADO
```sql
CREATE TABLE CONSUMO_FATURADO (
    id_consumo         INTEGER PRIMARY KEY,
    id_fatura         INTEGER,
    tipo_consumo      VARCHAR(50), -- TE, TUSD
    quantidade_kwh    INTEGER,
    tarifa_unitaria   DECIMAL(10,6),
    valor_total       DECIMAL(15,2),
    FOREIGN KEY (id_fatura) REFERENCES FATURA(id_fatura)
);
```

### 2.2 ENERGIA_COMPENSADA
```sql
CREATE TABLE ENERGIA_COMPENSADA (
    id_compensacao    INTEGER PRIMARY KEY,
    id_fatura         INTEGER,
    tipo_compensacao  VARCHAR(50), -- TE, TUSD, MPT, OPT
    quantidade_kwh    INTEGER,
    tarifa_unitaria   DECIMAL(10,6),
    valor_total       DECIMAL(15,2),
    FOREIGN KEY (id_fatura) REFERENCES FATURA(id_fatura)
);
```

### 2.3 BANDEIRA_TARIFARIA
```sql
CREATE TABLE BANDEIRA_TARIFARIA (
    id_bandeira       INTEGER PRIMARY KEY,
    id_fatura         INTEGER,
    tipo_bandeira     VARCHAR(50), -- Verde, Amarela, Vermelha P1, Vermelha P2
    periodo_inicio    DATE,
    periodo_fim       DATE,
    quantidade_kwh    INTEGER,
    valor_unitario    DECIMAL(10,6),
    valor_total       DECIMAL(15,2),
    FOREIGN KEY (id_fatura) REFERENCES FATURA(id_fatura)
);
```

## 3. Tabelas de Tributos e Encargos

### 3.1 TRIBUTOS
```sql
CREATE TABLE TRIBUTOS (
    id_tributo        INTEGER PRIMARY KEY,
    id_fatura         INTEGER,
    tipo_tributo      VARCHAR(20), -- ICMS, PIS, COFINS
    base_calculo      DECIMAL(15,2),
    aliquota         DECIMAL(6,4),
    valor            DECIMAL(15,2),
    FOREIGN KEY (id_fatura) REFERENCES FATURA(id_fatura)
);
```

### 3.2 ENCARGOS
```sql
CREATE TABLE ENCARGOS (
    id_encargo        INTEGER PRIMARY KEY,
    id_fatura         INTEGER,
    tipo_encargo      VARCHAR(50), -- Iluminação Pública, Contribuição, etc
    quantidade        INTEGER,
    valor_unitario    DECIMAL(10,2),
    valor_total       DECIMAL(15,2),
    FOREIGN KEY (id_fatura) REFERENCES FATURA(id_fatura)
);
```

## 4. Tabelas de Histórico e Controle

### 4.1 HISTORICO_CONSUMO
```sql
CREATE TABLE HISTORICO_CONSUMO (
    id_historico      INTEGER PRIMARY KEY,
    id_unidade        INTEGER,
    mes_referencia    DATE,
    consumo_kwh       INTEGER,
    demanda_kw        DECIMAL(10,2),
    dias_faturados    INTEGER,
    FOREIGN KEY (id_unidade) REFERENCES UNIDADE_CONSUMIDORA(id_unidade)
);
```

### 4.2 SALDO_GERACAO
```sql
CREATE TABLE SALDO_GERACAO (
    id_saldo          INTEGER PRIMARY KEY,
    id_fatura         INTEGER,
    saldo_anterior_kwh INTEGER,
    geracao_mes_kwh   INTEGER,
    credito_utilizado_kwh INTEGER,
    saldo_final_kwh   INTEGER,
    data_expiracao    DATE,
    FOREIGN KEY (id_fatura) REFERENCES FATURA(id_fatura)
);
```

## 5. Observações Importantes

1. **Compatibilidade entre Distribuidoras:**
   - A estrutura permite mapear diferentes nomenclaturas para o mesmo conceito
   - Campos específicos de cada distribuidora podem ser adicionados em tabelas auxiliares

2. **Controle de Versão:**
   - Recomenda-se manter histórico de alterações na estrutura
   - Importante documentar regras de negócio específicas de cada distribuidora

3. **Relacionamentos:**
   - Todas as tabelas principais estão conectadas através de chaves estrangeiras
   - Permite rastreabilidade completa das informações

4. **Campos de Auditoria Sugeridos:**
   - data_criacao
   - data_atualizacao
   - usuario_criacao
   - usuario_atualizacao

5. **Índices Recomendados:**
   - Número da unidade consumidora
   - Chave NFe
   - Datas de referência
   - CPF/CNPJ do titular

6. **Validações Importantes:**
   - Consistência entre valores calculados
   - Verificação de períodos de leitura
   - Validação de alíquotas por estado
   - Conferência de saldos de geração

7. **Campos de Totalização:**
   - Valores devem ser calculados e armazenados para conferência
   - Manter histórico de recálculos quando necessário

## 6. Exemplos de Consultas Úteis

```sql
-- Consumo médio por unidade
SELECT
    uc.numero_cliente,
    AVG(l.consumo_kwh) as consumo_medio
FROM UNIDADE_CONSUMIDORA uc
JOIN FATURA f ON f.id_unidade = uc.id_unidade
JOIN LEITURA l ON l.id_fatura = f.id_fatura
GROUP BY uc.numero_cliente;

-- Compensação de energia por período
SELECT
    uc.numero_cliente,
    SUM(ec.quantidade_kwh) as total_compensado,
    SUM(ec.valor_total) as valor_compensado
FROM UNIDADE_CONSUMIDORA uc
JOIN FATURA f ON f.id_unidade = uc.id_unidade
JOIN ENERGIA_COMPENSADA ec ON ec.id_fatura = f.id_fatura
WHERE f.data_emissao BETWEEN :data_inicio AND :data_fim
GROUP BY uc.numero_cliente;
```

## 7. Considerações para Implementação

1. **Segurança:**
   - Implementar controle de acesso por perfil
   - Criptografar dados sensíveis
   - Manter logs de alterações

2. **Performance:**
   - Particionar tabelas grandes por data
   - Criar índices específicos para consultas frequentes
   - Implementar estratégia de arquivamento

3. **Integridade:**
   - Implementar triggers para validações complexas
   - Manter consistência entre valores calculados
   - Validar regras de negócio específicas

4. **Expansão:**
   - Estrutura permite adição de novos campos
   - Possibilidade de incluir novas distribuidoras
   - Flexibilidade para novos tipos de tarifação

# Representação visual

Diagrama de entidade-relacionamento (ER) que ilustre a estrutura de dados proposta para armazenar as informações das contas de energia, mostrando as tabelas principais, os relacionamentos entre elas e os principais campos de cada entidade.

In [ ]:
#mermaid https://www.mermaidchart.com/

erDiagram
    DISTRIBUIDORA ||--o{ UNIDADE_CONSUMIDORA : possui
    UNIDADE_CONSUMIDORA ||--o{ FATURA : gera
    FATURA ||--|{ LEITURA : registra
    FATURA ||--|{ CONSUMO_FATURADO : contem
    FATURA ||--|{ ENERGIA_COMPENSADA : possui
    FATURA ||--|{ BANDEIRA_TARIFARIA : aplica
    FATURA ||--|{ TRIBUTOS : inclui
    FATURA ||--|{ ENCARGOS : cobra
    UNIDADE_CONSUMIDORA ||--o{ HISTORICO_CONSUMO : mantem
    FATURA ||--|{ SALDO_GERACAO : controla

    DISTRIBUIDORA {
        int id_distribuidora PK
        string nome
        string cnpj
        string inscricao_estadual
    }

    UNIDADE_CONSUMIDORA {
        int id_unidade PK
        int id_distribuidora FK
        string numero_cliente
        string grupo_subgrupo
        string classe_subclasse
        string tipo_fornecimento
        string filial_mult
        string filial_sap
        string endereco
        string cidade
        string estado
    }

    FATURA {
        int id_fatura PK
        int id_unidade FK
        string numero_fatura
        string serie_fatura
        date data_emissao
        date data_vencimento
        date data_leitura_atual
        date data_leitura_anterior
        int dias_faturados
        decimal valor_total
        string chave_nfe
    }

    LEITURA {
        int id_leitura PK
        int id_fatura FK
        int leitura_anterior
        int leitura_atual
        int consumo_kwh
        decimal demanda_medida
        decimal demanda_faturada
    }

    CONSUMO_FATURADO {
        int id_consumo PK
        int id_fatura FK
        string tipo_consumo
        int quantidade_kwh
        decimal tarifa_unitaria
        decimal valor_total
    }

    ENERGIA_COMPENSADA {
        int id_compensacao PK
        int id_fatura FK
        string tipo_compensacao
        int quantidade_kwh
        decimal tarifa_unitaria
        decimal valor_total
    }

    BANDEIRA_TARIFARIA {
        int id_bandeira PK
        int id_fatura FK
        string tipo_bandeira
        date periodo_inicio
        date periodo_fim
        int quantidade_kwh
        decimal valor_unitario
        decimal valor_total
    }

    TRIBUTOS {
        int id_tributo PK
        int id_fatura FK
        string tipo_tributo
        decimal base_calculo
        decimal aliquota
        decimal valor
    }

    ENCARGOS {
        int id_encargo PK
        int id_fatura FK
        string tipo_encargo
        int quantidade
        decimal valor_unitario
        decimal valor_total
    }

    HISTORICO_CONSUMO {
        int id_historico PK
        int id_unidade FK
        date mes_referencia
        int consumo_kwh
        decimal demanda_kw
        int dias_faturados
    }

    SALDO_GERACAO {
        int id_saldo PK
        int id_fatura FK
        int saldo_anterior_kwh
        int geracao_mes_kwh
        int credito_utilizado_kwh
        int saldo_final_kwh
        date data_expiracao
    }

1aae7f8e-ee38-4a4e-8e37-e53d1ebb86c4.svg

Diagrama de fluxo de dados que mostre o caminho das informações no sistema, desde o cadastro inicial até o processamento final da fatura.

In [ ]:
#mermaid https://www.mermaidchart.com/

flowchart TB
    subgraph Entrada["Entrada de Dados"]
        A1[Cadastro da Distribuidora]
        A2[Cadastro da Unidade Consumidora]
        A3[Importação da Fatura]
    end

    subgraph Processamento["Processamento Principal"]
        B1[Registro de Leituras]
        B2[Cálculo de Consumo]
        B3[Aplicação de Tarifas]
        B4[Processamento de Compensação]
        B5[Aplicação de Bandeiras]
    end

    subgraph Tributação["Cálculos Tributários"]
        C1[Base de Cálculo]
        C2[Aplicação de Alíquotas]
        C3[Cálculo de Encargos]
    end

    subgraph Consolidação["Consolidação e Controle"]
        D1[Atualização de Histórico]
        D2[Gestão de Saldos]
        D3[Totalização da Fatura]
        D4[Validações Finais]
    end

    subgraph Armazenamento["Armazenamento em Tabelas"]
        E1[(DISTRIBUIDORA)]
        E2[(UNIDADE_CONSUMIDORA)]
        E3[(FATURA)]
        E4[(LEITURA)]
        E5[(CONSUMO_FATURADO)]
        E6[(ENERGIA_COMPENSADA)]
        E7[(BANDEIRA_TARIFARIA)]
        E8[(TRIBUTOS)]
        E9[(ENCARGOS)]
        E10[(HISTORICO_CONSUMO)]
        E11[(SALDO_GERACAO)]
    end

    %% Fluxos de Entrada
    A1 --> E1
    A2 --> E2
    A3 --> E3

    %% Fluxos de Processamento
    E3 --> B1
    B1 --> E4
    B1 --> B2
    B2 --> B3
    B3 --> E5
    B3 --> B4
    B4 --> E6
    B3 --> B5
    B5 --> E7

    %% Fluxos de Tributação
    B3 --> C1
    C1 --> C2
    C2 --> E8
    C1 --> C3
    C3 --> E9

    %% Fluxos de Consolidação
    B2 --> D1
    D1 --> E10
    B4 --> D2
    D2 --> E11
    E5 --> D3
    E6 --> D3
    E7 --> D3
    E8 --> D3
    E9 --> D3
    D3 --> D4
    D4 --> E3

    style Entrada fill:#e1f5fe,stroke:#01579b
    style Processamento fill:#f3e5f5,stroke:#4a148c
    style Tributação fill:#fff3e0,stroke:#e65100
    style Consolidação fill:#e8f5e9,stroke:#1b5e20
    style Armazenamento fill:#fafafa,stroke:#212121

    classDef processo fill:#fff,stroke:#333,stroke-width:2px
    classDef database fill:#eceff1,stroke:#263238,stroke-width:2px
    class A1,A2,A3,B1,B2,B3,B4,B5,C1,C2,C3,D1,D2,D3,D4 processo
    class E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,E11 database

71c7b00d-c555-4a1f-8484-0bd3c42f1608.svg